In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# ===============================================================
#  COMPARAÇÃO MÉTRICAS — REAL vs. SIMULADO (UM PLAY NO COLAB)
# ===============================================================
# CONFIGURAÇÃO  ──────────────────────────────────────────────────
X_FILE       = "/content/drive/MyDrive/Temp/FULLSQUARE/sent_data_tuple.npy"
Y_REAL_FILE  = "/content/drive/MyDrive/Temp/FULLSQUARE/y_real_1p2m.npy"
Y_HAT_FILE   = "/content/drive/MyDrive/Temp/simulacao_1p2m/y_hat_1.20m.npy"
# ───────────────────────────────────────────────────────────────

import numpy as np, matplotlib.pyplot as plt
from scipy import stats

# ---------- 0) NumPy ou CuPy -----------------------------------
try:
    import cupy as cp
    xp, gpu_enabled = cp, True
    print("CuPy backend → métricas na GPU")
except ImportError:
    xp, gpu_enabled = np, False
    print("NumPy backend → métricas na CPU")

# ---------- 1) Funções de métrica (mesmas do treino) ------------
def safe_log10(x, eps=1e-12): return xp.log10(xp.maximum(x, eps))
def safe_log2(x, eps=1e-12):  return xp.log2(xp.maximum(x, eps))

def calculate_evm(ref, test):
    rc = xp.asarray(ref[:,0] + 1j*ref[:,1])
    tc = xp.asarray(test[:,0] + 1j*test[:,1])
    evm = xp.sqrt(xp.mean(xp.abs(tc-rc)**2) / xp.mean(xp.abs(rc)**2))
    return float(evm*100), float(20*safe_log10(evm))

def calculate_snr(ref, test):
    rc = xp.asarray(ref[:,0] + 1j*ref[:,1])
    tc = xp.asarray(test[:,0] + 1j*test[:,1])
    snr = xp.mean(xp.abs(rc)**2) / xp.mean(xp.abs(rc-tc)**2)
    return float(10*safe_log10(snr))

def calculate_noise_power(sent, recv):
    return float(xp.mean(xp.sum((xp.asarray(recv)-xp.asarray(sent))**2, axis=1)))

def compute_basic_metrics(x, y):
    err = xp.asarray(x) - xp.asarray(y)
    mse  = float(xp.mean(xp.sum(err**2, axis=1)))
    rmse = float(xp.sqrt(mse))
    mae  = float(xp.mean(xp.linalg.norm(err, axis=1)))
    normx, normy = xp.linalg.norm(x), xp.linalg.norm(y)
    cos  = 0.0 if normx==0 or normy==0 else float(xp.dot(x.ravel(), y.ravel())/(normx*normy))
    x_cpu = cp.asnumpy(x) if gpu_enabled else x
    y_cpu = cp.asnumpy(y) if gpu_enabled else y
    corr = float(np.corrcoef(x_cpu.ravel(), y_cpu.ravel())[0,1])
    return mse, rmse, mae, cos, corr

def calculate_psnr(mse, peak=1.0):
    return float(20*np.log10(peak) - 10*np.log10(mse))

def _hist2d(arr, bins=200, lim=1.5):
    H, _ = np.histogramdd(arr, bins=bins, range=[[-lim,lim],[-lim,lim]], density=True)
    return H + 1e-12

def calculate_kl_jsd(real, pred):
    P, Q = _hist2d(real), _hist2d(pred)
    kl  = float(np.sum(P*np.log(P/Q)))
    M   = 0.5*(P+Q)
    jsd = float(0.5*np.sum(P*np.log2(P/M)) + 0.5*np.sum(Q*np.log2(Q/M)))
    return kl, jsd

def calculate_ks_test(a, b):
    if gpu_enabled: a, b = cp.asnumpy(a), cp.asnumpy(b)
    ks, p = stats.ks_2samp(a.ravel(), b.ravel())
    return float(ks), float(p)

# ---------- 2) Carrega dados -----------------------------------
X      = np.load(X_FILE)
Y_real = np.load(Y_REAL_FILE)
Y_hat  = np.load(Y_HAT_FILE)
print("Shapes -> X:", X.shape, "| Y_real:", Y_real.shape, "| Y_hat:", Y_hat.shape)

# ---------- 3) Métricas ----------------------------------------
evm_real_pct, _ = calculate_evm(X, Y_real)
evm_hat_pct,  _ = calculate_evm(X, Y_hat)
snr_real        = calculate_snr(X, Y_real)
snr_hat         = calculate_snr(X, Y_hat)
noise_real      = calculate_noise_power(X, Y_real)
noise_hat       = calculate_noise_power(X, Y_hat)
kl_div, jsd     = calculate_kl_jsd(Y_real, Y_hat)
mse, rmse, mae, cos, corr = compute_basic_metrics(X, Y_hat)
psnr            = calculate_psnr(mse)
ks_I, p_I = calculate_ks_test(Y_real[:,0], Y_hat[:,0])
ks_Q, p_Q = calculate_ks_test(Y_real[:,1], Y_hat[:,1])

# ---------- 4) Relatório ---------------------------------------
print("\n=== COMPARAÇÃO Real (Y_real) vs. Simulado (Y_hat) ===\n")
print(f"EVM real       : {evm_real_pct:7.3f} %")
print(f"EVM simulado   : {evm_hat_pct:7.3f} %   ΔEVM = {evm_hat_pct-evm_real_pct:+.3f} %")
print(f"SNR real       : {snr_real:7.3f} dB")
print(f"SNR simulado   : {snr_hat:7.3f} dB   ΔSNR = {snr_hat-snr_real:+.3f} dB")
print(f"ΔPotência ruído: {noise_hat-noise_real:+.3e}")
print(f"MSE            : {mse:.4e}")
print(f"RMSE           : {rmse:.4e}")
print(f"MAE            : {mae:.4e}")
print(f"PSNR           : {psnr:.2f} dB")
print(f"CosSim         : {cos:.5f}")
print(f"Correlação     : {corr:.5f}")
print("\n--- Divergência de distribuição ---")
print(f"KL   (Y_real‖Ŷ) : {kl_div:.4f}")
print(f"JSD  (Y_real,Ŷ) : {jsd:.4f}")
print("\n--- KS-test ---")
print(f"I: estat={ks_I:.4f},  p={p_I:.4f}")
print(f"Q: estat={ks_Q:.4f},  p={p_Q:.4f}")
